## Challenge: Crowd (Object) Counting

<img src="http://personal.ie.cuhk.edu.hk/~ccloy/images/shopping_mall_annotated.jpg" title="Crowd Counting" width="30%"/>

Para esta sección final se evaluará todo lo que han aprendido a través de un desafío en donde puedan competir y medir sus resultados *in time* en la plataforma de __[Kaggle](https://www.kaggle.com/)__. El problema y todo su detalle puede ser encontrado en la página de la competencia a través del siguiente link:


<center><H2> __[Competencia Object Counting](https://www.kaggle.com/t/59c93ca0e8ae47999f9287a5751d6402)__ </H2></center>



Para esto deberán crearse una cuenta en la plataforma *Kaggle* y subir sus respuestas a ésta. Por favor crearse nombres que sean fácil identificar después para saber quién fue quién, sino no podrán tener la nota (o en el correo de entrega ponen cual es su nombre de usuario en la competencia).

*Nota: El puntaje que entregará al estar participando en la competencia (menos de 2 puntos al *benchmark* que se presenta) y el lugar obtenido se presenta en la página de Kaggle.*

> Las entregas en *csv* pueden ser generadas de la siguiente manera:
```python
import pandas as pd
d = {'id': test_ids, 'count': prediction_test}
entrega = pd.DataFrame(data=d,columns=['id','count'])
entrega.to_csv('mysubmission.csv', index=False)
```

> Los archivos binarios de numpy pueden ser cargados facilmente con el siguiente comando:
```python
import numpy as np
dato = np.load('binary_file.npy')
```

> Las imagenes en python pueden ser cargadas con la librería Pillow y transformadas a numpy array con el siguiente comando:
```python
from PIL import Image
img = Image.open("path/to/image.jpg or png")
np.asarray(img)
```

In [1]:
import numpy as np
import pandas as pd
from PIL import Image

import matplotlib.pyplot as plt
%matplotlib inline

from keras.layers import *
from keras.models import Model,Sequential
from keras.optimizers import SGD
from keras.utils import to_categorical

/home/fefi/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Datos

In [ ]:
img = Image.open("../frames_train/seq_000401.jpg")
np.asarray(img)

In [ ]:
plt.imshow(img)
plt.show()

In [2]:
train = np.load('../images_train.npy')

In [15]:
test = np.load('../images_test.npy')

In [5]:
df_y = pd.read_csv("../labels_train.csv")

In [6]:
df_y.describe()

,id,count
count,1600.00000,1600.000000
mean,1200.50000,31.070625
std,462.02453,6.992134
min,401.00000,13.000000
25%,800.75000,26.000000
50%,1200.50000,30.000000
75%,1600.25000,35.000000
max,2000.00000,50.000000


In [7]:
df_y.head()

,id,count
0,401,29
1,402,30
2,403,31
3,404,26
4,405,26


In [8]:
print(train.shape)
print(type(train))

(1600, 480, 640, 3)
<class 'numpy.ndarray'>


In [16]:
print(test.shape)
print(type(test))

(400, 480, 640, 3)
<class 'numpy.ndarray'>


In [3]:
idx_test = np.random.choice(np.arange(len(train)), 600, replace=False)
mask_test = np.zeros(len(train),dtype=bool) 
mask_test[idx_test] = True 

X_train = train[~mask_test][:800]
X_val = train[mask_test][:400]

In [10]:
y_train = df_y['count'].values
Y_train = y_train[~mask_test][:800]
Y_val = y_train[mask_test][:400]

In [4]:
X_train = X_train/255.0

In [11]:
Y_train = to_categorical(Y_train, num_classes=51)

In [12]:
#norm
X_val = X_val/255.0

In [13]:
Y_val = to_categorical(Y_val, num_classes=51)

In [18]:
X_test = test/255.0

MemoryError: 

In [19]:
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)
print(X_test.shape)

(800, 480, 640, 3)
(800, 51)
(400, 480, 640, 3)
(400, 51)


NameError: name 'X_test' is not defined

In [23]:
Y_train.shape

(800, 51)

## Arquitectura

In [24]:
model = Sequential()
model.add(Conv2D(32,(3,3),padding='same',input_shape=X_train.shape[1:],activation='tanh'))
#aux.add(LeakyReLU(alpha=0.3))

model.add(BatchNormalization())

model.add(MaxPooling2D(2,2))

model.add(Conv2D(64,(3,3),padding='same',activation='tanh')) 
#aux.add(LeakyReLU(alpha=0.3))
model.add(BatchNormalization())

model.add(MaxPooling2D(2,2)) 

#tercera capa
#aux.add(Conv2D(128,(3,3),padding='same',kernel_regularizer=weight,activation='tanh')) 
#aux.add(BatchNormalization())
#aux.add(MaxPooling2D(2,2)) 

#aux.add(BatchNormalization())
model.add(Conv2D(1,(1,1),padding='same',activation='sigmoid'))

#model.add(AveragePooling2D(2,2)) 

#aux.add(UpSampling2D((3,3)))
#aux.add(UpSampling2D((3,3)))

#esos 5 a 1...
#model=Lambda(myFunc,output_shape=(1,))(aux.output)   

model.add(Flatten())
#model.add(Dense(1024,activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(51,activation='relu'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 480, 640, 32)      896       
_________________________________________________________________
batch_normalization_3 (Batch (None, 480, 640, 32)      128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 240, 320, 32)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 240, 320, 64)      18496     
_________________________________________________________________
batch_normalization_4 (Batch (None, 240, 320, 64)      256       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 120, 160, 64)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 120, 160, 1)       65        
__________

In [ ]:
opt = SGD(lr=0.1, momentum=0.9)
model.compile( optimizer='adam', loss='mse') #mse
model.fit(X_train, Y_train,batch_size=8,epochs=10,validation_data=(X_val, Y_val),verbose=1)

Train on 800 samples, validate on 400 samples
Epoch 1/10


In [ ]:
model2 = Sequential()
model2.add(Conv2D(32, (3, 3), padding='same',input_shape=X_train.shape[1:],activation='relu'))
model2.add(Conv2D(32, (3, 3),padding='same',activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Conv2D(64, (3, 3),padding='same',activation='relu'))
model2.add(Conv2D(64, (3, 3),padding='same',activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Flatten())
model2.add(Dense(1024,activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(2,activation='softmax'))
model2.summary()


In [ ]:
#train it
#optimizer_ = SGD(lr=0.01,momentum=0.9)
model2.compile(loss='mse', optimizer="adam", metrics=['accuracy'])
model2.fit(X_train, Y_train, batch_size=128,epochs=15,verbose=1, validation_data=(X_val,Y_val))